In [2]:
%cd ~/comp_mut
%pwd

/mnt/storage7/gary/comp_mut


'/mnt/storage7/gary/comp_mut'

In [3]:
import csv
import json
import os.path
from tqdm import tqdm
from collections import defaultdict, Counter
import pandas as pd

In [4]:
# Files
# Change these!
# metafile = 'tb_data_18_02_2021.csv'
# jsondir = '/Users/jodyphelan//projects/results/'
# tbdbfile = 'tbdb_altered.csv'


metafile = '../metadata/tb_data_18_02_2021.csv'
jsondir = '/mnt/storage7/jody/tb_ena/tbprofiler/freebayes/results/'
# tbdbfile = 'tbdb_altered.csv'
tbdbfile = "../tbdb/tbdb.csv"
comp_mutfile = 'compensatory_mutations.csv'

In [5]:
def resolve_lineages(data):

    data = dict(data)
    while True:
        improved = False
        for child in sorted(data,reverse= True,key=lambda x : len(x)):
            parent = ".".join(child.split(".")[:-1])
            if any([parent == x for x in data]):
                data[parent] = data[parent] + data[child] 
                del data[child]
                improved = True
                break
        if not improved:
            break
    return data

In [6]:
def json_file(jsondir, s):
    """ 
    Given a sample name as input it returns
    the full path to the json file
    """
    return f"{jsondir}/{s}.results.json"

In [7]:
os.path.isfile(metafile)

True

In [8]:
# Create a dictionary with the metadata
# Sample names as keys
meta = {}
for row in tqdm(csv.DictReader(open(metafile))):
    if os.path.isfile(json_file(jsondir, row['wgs_id'])):
        meta[row['wgs_id']] = row

32735it [00:00, 41572.53it/s]


In [9]:
# Create a simple list of samples
samples = list(meta.keys())
print(len(samples))

32718


In [10]:
compensatory_mutations = defaultdict(set)
for row in csv.DictReader(open(tbdbfile)):
#     if row['Confers'] != 'compensatory': continue
    if row['Gene'] != 'ahpC': continue
    compensatory_mutations[row['Drug']].add((row['Gene'],row['Mutation']))
    
compensatory_mutations

defaultdict(set,
            {'isoniazid': {('ahpC', 'c.-20C>T'),
              ('ahpC', 'c.-39C>T'),
              ('ahpC', 'c.-44T>A'),
              ('ahpC', 'c.-48G>A'),
              ('ahpC', 'c.-48G>T'),
              ('ahpC', 'c.-49T>G'),
              ('ahpC', 'c.-4A>G'),
              ('ahpC', 'c.-51G>T'),
              ('ahpC', 'c.-52C>T'),
              ('ahpC', 'c.-54C>T'),
              ('ahpC', 'c.-57C>T'),
              ('ahpC', 'c.-5G>A'),
              ('ahpC', 'c.-66G>A'),
              ('ahpC', 'c.-74G>A'),
              ('ahpC', 'c.-81C>T'),
              ('ahpC', 'p.Asp33Asn'),
              ('ahpC', 'p.Asp73His'),
              ('ahpC', 'p.Glu76Lys'),
              ('ahpC', 'p.Leu191Arg'),
              ('ahpC', 'p.Phe10Ile'),
              ('ahpC', 'p.Pro2Ser'),
              ('ahpC', 'p.Thr5Ile')}})

In [11]:
# samples = samples[:1000]

In [12]:
# Load mutation data using ('gene','change') as keys
# genes = set(target_df['gene'])
genes = ('ahpC', 'katG', 'fabG1')
mutation2sample = defaultdict(set)
sample2mutation = defaultdict(set)
resistance_mutations = defaultdict(set)
for s in tqdm(samples):
    data = json.load(open(json_file(jsondir, s)))
    
    if ';' in data['sublin']: continue
    
    meta[s]['drtype'] = data['drtype']
    meta[s]['sublin'] = data['sublin']
    
    for var in data['dr_variants'] + data['other_variants']:
        if var['gene'] not in genes: continue
        if var['freq'] < 0.7: continue
        if var['type']=='synonymous_variant': continue
        key = (var['gene'],var['change'])
        mutation2sample[key].add(s)
        sample2mutation[s].add(key)
        if "drugs" in var:
            for d in var["drugs"]:
                if key in compensatory_mutations[d["drug"]]: continue
                resistance_mutations[d["drug"]].add(key)

100%|██████████| 32718/32718 [00:08<00:00, 3777.36it/s]


In [13]:
len(mutation2sample)

1111

In [14]:
len(sample2mutation)

19381

In [15]:
len(resistance_mutations)

2

In [16]:
drug = 'isoniazid'
potential_resistance_mutations = set()
for s in tqdm(samples):
    comp_var = [var for var in sample2mutation[s] if var in compensatory_mutations[drug]]
    res_var = [var for var in sample2mutation[s] if var in resistance_mutations[drug]]
    other_vars = [var for var in sample2mutation[s] if var not in compensatory_mutations[drug] and \
                  var not in resistance_mutations[drug]]
    # If there is at least one comp variant and there are no resistance variants
    if len(comp_var)>0 and len(res_var)==0:
        # If there are no 'other' variants print the sample and the comp variants
        if len(other_vars)==0:
            print("Sample, comp. variant")
            print(s,comp_var)
        # Store the 'other' vars as potential resistance variants
        for var in other_vars:
            potential_resistance_mutations.add(var)


100%|██████████| 32718/32718 [00:00<00:00, 314210.50it/s]

Sample, comp. variant
ERR2516197 [('ahpC', 'p.Leu191Arg')]
Sample, comp. variant
ERR2516199 [('ahpC', 'c.-52C>T')]
Sample, comp. variant
ERR2179658 [('ahpC', 'c.-81C>T')]
Sample, comp. variant
SAMN03246659 [('ahpC', 'c.-81C>T')]
Sample, comp. variant
SRR1723651 [('ahpC', 'c.-81C>T')]
Sample, comp. variant
SRR6824287 [('ahpC', 'c.-81C>T')]
Sample, comp. variant
SRR8651572 [('ahpC', 'c.-48G>A')]
Sample, comp. variant
ERR216923 [('ahpC', 'c.-48G>A')]
Sample, comp. variant
ERR3148230 [('ahpC', 'p.Asp73His')]
Sample, comp. variant
ERR3148166 [('ahpC', 'p.Asp73His')]
Sample, comp. variant
ERR3148207 [('ahpC', 'p.Asp73His')]
Sample, comp. variant
ERR550722 [('ahpC', 'c.-52C>T')]
Sample, comp. variant
ERR553062 [('ahpC', 'c.-52C>T')]
Sample, comp. variant
ERR553076 [('ahpC', 'c.-48G>A')]
Sample, comp. variant
SAMEA2534034 [('ahpC', 'c.-57C>T')]
Sample, comp. variant
ERR553226 [('ahpC', 'c.-81C>T')]
Sample, comp. variant
SRR2101249 [('ahpC', 'c.-52C>T')]
Sample, comp. variant
ERR2446396 [('ahpC

In [17]:
potential_resistance_mutations

{('ahpC', 'c.-142G>A'),
 ('ahpC', 'p.Leu191Pro'),
 ('fabG1', 'c.-102G>A'),
 ('fabG1', 'c.-179C>T'),
 ('fabG1', 'c.-223_-222insGC'),
 ('katG', 'c.-1T>C'),
 ('katG', 'c.-634_489del'),
 ('katG',
  'c.1042_1128delNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNA'),
 ('katG', 'c.1120_*453del'),
 ('katG', 'c.1438_1440delGCG'),
 ('katG', 'c.2223A>C'),
 ('katG', 'c.2223A>G'),
 ('katG', 'c.680_685delTGATCT'),
 ('katG', 'c.718_*16215del'),
 ('katG', 'c.984_998delGGACAACAGTTTCCT'),
 ('katG', 'p.Ala122Val'),
 ('katG', 'p.Ala162Val'),
 ('katG', 'p.Ala312Glu'),
 ('katG', 'p.Ala312Val'),
 ('katG', 'p.Ala411Asp'),
 ('katG', 'p.Ala480Gln'),
 ('katG', 'p.Ala621Asp'),
 ('katG', 'p.Ala93Thr'),
 ('katG', 'p.Arg385Pro'),
 ('katG', 'p.Arg463Leu'),
 ('katG', 'p.Arg484His'),
 ('katG', 'p.Arg705Trp'),
 ('katG', 'p.Arg78Pro'),
 ('katG', 'p.Asn655Asp'),
 ('katG', 'p.Asp142Gly'),
 ('katG', 'p.Asp163Ala'),
 ('katG', 'p.Asp163Asn'),
 ('katG', 'p.Asp189Gly'),
 ('katG', 'p.Asp329A

In [205]:
def get_counts(meta,samples,col):
    return dict(Counter([meta[s][col] for s in samples]))

def get_meta_proportion(meta,samples,column,targets):
    tmp = get_counts(meta,samples,column)
    target_count = sum([tmp.get(c,0) for c in targets])
    return round(target_count/sum(tmp.values()), 3)

In [279]:
# Get stats for each variant
# tmp = set()
for var in potential_resistance_mutations: 
    # Get number of samples per potential resitance mutations
    samps = mutation2sample[var]

    if len(samps)<3: continue
#     tmp.add(var)
    resistant_dst_proportion = get_meta_proportion(meta,samps,drug,['1'])
    sensitive_geno_proportion = get_meta_proportion(meta,samps,'drtype',['Sensitive'])
    num_lins = len(set(get_counts(meta,samps,'sublin')))
    stats_dict = {'var': var, 
                  'n_samps' : len(samps), 
                  'dst_prop': resistant_dst_proportion, 
                  'dr_prop': sensitive_geno_proportion, 
                  'n_lins': num_lins}
#     print(var,len(samps),resistant_dst_proportion,sensitive_geno_proportion,num_lins)
    print(stats_dict)
    print('Lineage counts:')
    print(Counter(resolve_lineages(get_counts(meta,samps,'sublin'))))
    print()

{'var': ('katG', 'p.Gly169Ser'), 'n_samps': 7, 'dst_prop': 0.714, 'dr_prop': 0.0, 'n_lins': 2}
Lineage counts:
Counter({'lineage2.2.1': 5, 'lineage2.2.2': 2})

{'var': ('katG', 'p.Gly182Arg'), 'n_samps': 3, 'dst_prop': 0.333, 'dr_prop': 0.0, 'n_lins': 2}
Lineage counts:
Counter({'lineage4.6.1.2': 2, 'lineage2.2.1': 1})

{'var': ('katG', 'c.2223A>G'), 'n_samps': 3, 'dst_prop': 0.667, 'dr_prop': 0.0, 'n_lins': 3}
Lineage counts:
Counter({'lineage4.1.2.1.1': 1, 'lineage2.2.1': 1, 'lineage4.3.4.1': 1})

{'var': ('katG', 'p.Trp90Arg'), 'n_samps': 3, 'dst_prop': 1.0, 'dr_prop': 0.0, 'n_lins': 3}
Lineage counts:
Counter({'lineage4.4.2': 1, 'lineage5.1.2': 1, 'lineage2.2.1': 1})

{'var': ('katG', 'p.Trp505*'), 'n_samps': 4, 'dst_prop': 0.5, 'dr_prop': 0.0, 'n_lins': 3}
Lineage counts:
Counter({'lineage3': 2, 'lineage2.2.1': 1, 'lineage4.3.3': 1})

{'var': ('katG', 'p.Ala312Glu'), 'n_samps': 4, 'dst_prop': 0.5, 'dr_prop': 0.0, 'n_lins': 1}
Lineage counts:
Counter({'lineage2.2.1': 4})

{'var': (